In [1]:
!pip install selenium

In [2]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import pandas as pd

In [3]:
driver = webdriver.Chrome()

In [4]:
# driver = webdriver.Chrome(executable_path=r"C:\Users\hp\Desktop\chromedriver.exe")

In [7]:
url="https://www.volunteertoronto.ca/networking/search_results.asp?__ASPVIEWSTATE=eab7e4140b6b55d76e8f93178b9fea325da4cb9d06300622eab549e266650700ead743cdf176272c91a4e0577bd13dfdd6aea37c03e265894e365737e3bebc8e0507cf74b6fa72d7e0f9f4f4e69492eb57d167638ab7fa71c9b3b33bf45a00793d1abb6737733f0d6bc443d95cd28c42ef7571297e97879a333bd97d40fc28658a93cd7a9d32a7919005405404094d1d54ce474618eb674392bee6aa36a2e30abc835eb9c1b04a88802cf49de46ee27f8c0b715d8646f38c680126d8bf9d80338b1caa8c8f1c343ae11ab6cdde6189e168b617951aa9c8b25632870de1a964a6125abb0f9bf592a27a822637a2cd403fa3736d90662c8f4aa7b1441a180a10d468d1fe0c"
driver.get(url)

In [8]:
html_content = requests.get(url).text
soup = BeautifulSoup(html_content, "lxml")

### table

In [9]:
data=[]
table = soup.find('table', attrs={'class':'datagrid1'})
rows = table.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

In [11]:
print(data[:5])

[['Position', 'Primary Category', 'Organization', 'Date Posted'], [], ['THIS SATURDAY - Contactless Delivery Driver Volunteers\r\n\t\t\t\tToronto, Ontario', 'COVID-19 Response', 'Hand Up Toronto', '2020-06-23'], [], ['Marketing and Communications Manager/Coordinator for a Registered Charity\r\n\t\t\t\tToronto, Ontario', 'COVID-19 Response', 'Hand Up from Hunger Toronto', '2020-06-23']]


In [12]:
Position=[]
Location=[]
Primary_Category=[]
Organization=[]
Date_Posted=[]
for i in range(1,len(data)):
    if len(data[i]) == 4:
        x = data[i][0].split('\r\n\t\t\t\t')
        Position.append(x[0])
        if len(x)>1:
            Location.append(x[1])
        else:
            Location.append('nan')
        Primary_Category.append(data[i][1])
        Organization.append(data[i][2])
        Date_Posted.append(data[i][3])

### Websites

In [13]:
webs=table.findAll('a')
prefix_url = 'https://www.volunteertoronto.ca/networking/'

In [14]:
position = []
href=[]
for link in webs:
    if len(link.get("href")) > 2 and len(link.get("href")) < 100:
        position.append(link.text)
        href.append(link.get("href"))
#         print("Inner Text: {}".format(link.text))
#         print("href: {}".format(link.get("href")))

Inner Text: THIS SATURDAY - Contactless Delivery Driver Volunteers
href: apply_now.aspx?view=2&id=608778
Inner Text: Marketing and Communications Manager/Coordinator for a Registered Charity
href: apply_now.aspx?view=2&id=608671
Inner Text: Mover
href: apply_now.aspx?view=2&id=516258
Inner Text: Driver
href: apply_now.aspx?view=2&id=556205
Inner Text: Scarborough Drop In Food for Families 
href: apply_now.aspx?view=2&id=225152
Inner Text: COVID-19: Remote Administrative Support Volunteer
href: apply_now.aspx?view=2&id=608457
Inner Text: Retired Persons Opportunity
href: apply_now.aspx?view=2&id=561521
Inner Text: COVID-19: Food Packing Volunteer
href: apply_now.aspx?view=2&id=608455
Inner Text: COVID-19: Driver
href: apply_now.aspx?view=2&id=608454
Inner Text: Meal/Grocery Delivery Volunteer
href: apply_now.aspx?view=2&id=608278
Inner Text: COVID-19: Online Tutor
href: apply_now.aspx?view=2&id=608231
Inner Text: Volunteer Delivery Driver
href: apply_now.aspx?view=2&id=607880
Inner Text

In [21]:
Websites=[]
for i in range(len(href)):
    print("Position: {}".format(position[i]))
    Websites.append(prefix_url+href[i])
    print("Website: {}".format(Websites[i]))

Position: THIS SATURDAY - Contactless Delivery Driver Volunteers
Website: https://www.volunteertoronto.ca/networking/apply_now.aspx?view=2&id=608778
Position: Marketing and Communications Manager/Coordinator for a Registered Charity
Website: https://www.volunteertoronto.ca/networking/apply_now.aspx?view=2&id=608671
Position: Mover
Website: https://www.volunteertoronto.ca/networking/apply_now.aspx?view=2&id=516258
Position: Driver
Website: https://www.volunteertoronto.ca/networking/apply_now.aspx?view=2&id=556205
Position: Scarborough Drop In Food for Families 
Website: https://www.volunteertoronto.ca/networking/apply_now.aspx?view=2&id=225152
Position: COVID-19: Remote Administrative Support Volunteer
Website: https://www.volunteertoronto.ca/networking/apply_now.aspx?view=2&id=608457
Position: Retired Persons Opportunity
Website: https://www.volunteertoronto.ca/networking/apply_now.aspx?view=2&id=561521
Position: COVID-19: Food Packing Volunteer
Website: https://www.volunteertoronto.ca

### Job Description

In [184]:
Description=[]
for link in Websites:
    html_content = requests.get(link).text
    soup = BeautifulSoup(html_content, "lxml")
    des=soup.find_all('tr')
    idx_des = [i for i in range(len(des)) if des[i].find('b',string='Description & Details') != None]
    idx=des[idx_des[-1]+1]
    Description.append(idx.get_text())

### Output CSV file

In [185]:
df = pd.DataFrame({'Position':Position,'Location':Location, 
                   'Primary_Category':Primary_Category,
                 'Organization':Organization,
                 'Date_Posted':Date_Posted,
                  "Websites": Websites,
                  "Description": Description}) 

In [188]:
df.to_csv('toronto_volunteer.csv', index=False, encoding='utf-8_sig')

In [189]:
df

,Position,Location,Primary_Category,Organization,Date_Posted,Websites,Description
0,THIS SATURDAY - Contactless Delivery Driver Vo...,"Toronto, Ontario",COVID-19 Response,Hand Up Toronto,2020-06-23,https://www.volunteertoronto.ca/networking/app...,"\n\nWe are looking for volunteer drivers, with..."
1,Marketing and Communications Manager/Coordinat...,"Toronto, Ontario",COVID-19 Response,Hand Up from Hunger Toronto,2020-06-23,https://www.volunteertoronto.ca/networking/app...,\n\nLooking for energetic marketing profession...
2,Mover,"Toronto, Ontario",COVID-19 Response,Shelter Movers,2020-06-22,https://www.volunteertoronto.ca/networking/app...,\n\nShelter Movers is a volunteer-based organi...
3,Driver,"Toronto, Ontario",COVID-19 Response,Shelter Movers - Toronto,2020-06-19,https://www.volunteertoronto.ca/networking/app...,\n\n\nShelter Movers provides moving and stora...
4,Scarborough Drop In Food for Families,"toronto, Ontario",COVID-19 Response,Good neighbours Drop in,2020-06-19,https://www.volunteertoronto.ca/networking/app...,\n\n \n \nWe are looking for volunteers who ca...
5,COVID-19: Remote Administrative Support Volunteer,"Etobicoke, Toronto, North York, Scarborough, O...",COVID-19 Response,Toronto Cares Initiative,2020-06-19,https://www.volunteertoronto.ca/networking/app...,\n\nToronto Cares is a small nonprofit organiz...
6,Retired Persons Opportunity,"toronto, Ontario",COVID-19 Response,st john the compassionate mission,2020-06-19,https://www.volunteertoronto.ca/networking/app...,\n\n \n \nManaging a successful store that hel...
7,COVID-19: Food Packing Volunteer,"Toronto, Ontario",COVID-19 Response,Toronto Cares Initiative,2020-06-19,https://www.volunteertoronto.ca/networking/app...,\n\nToronto Cares is a small nonprofit organiz...
8,COVID-19: Driver,"Etobicoke, Toronto, North York, Scarborough, O...",COVID-19 Response,Toronto Cares Initiative,2020-06-19,https://www.volunteertoronto.ca/networking/app...,\n\nToronto Cares is a small nonprofit organiz...
9,Meal/Grocery Delivery Volunteer,"Toronto, Ontario",COVID-19 Response,SPRINT Senior Care,2020-06-17,https://www.volunteertoronto.ca/networking/app...,\n\nAbout SPRINT Senior Care\nWe began caring ...
